# Tutorial 04: Human-in-the-Loop

In this tutorial, you'll learn how to pause agent execution for human review and approval before taking sensitive actions.

**What you'll learn:**
- **Interrupts**: Pausing graph execution
- **interrupt_before**: Static breakpoints at compile time
- **interrupt()**: Dynamic breakpoints at runtime
- **Command**: Resuming execution with human input
- **Approval workflows**: Review before action

By the end, you'll have an agent that asks for approval before executing sensitive tool calls.

## Why Human-in-the-Loop?

Agents are powerful but not infallible. Before an agent:
- Sends an email
- Makes a purchase
- Deletes data
- Calls an external API

You might want a human to review and approve the action.

### Common Patterns

1. **Approve/Reject**: Review tool calls before execution
2. **Edit State**: Modify agent's proposed action
3. **Provide Input**: Ask human for additional information
4. **Review Output**: Validate before returning to user

## Graph Visualization

![Human-in-the-Loop Graph](../docs/images/04-human-in-loop-graph.png)

The graph pauses before the `tools` node, allowing human review.

In [ ]:
# Setup
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Build a Tool-Calling Agent

First, let's create a ReAct agent like Tutorial 02:

In [ ]:
import json
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# Define a "sensitive" tool
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to a recipient.
    
    Args:
        to: Email address of recipient
        subject: Email subject line
        body: Email body content
    """
    # In production, this would actually send an email!
    return f"Email sent to {to} with subject: {subject}"

@tool
def get_weather(location: str) -> str:
    """Get weather for a location."""
    return f"Weather in {location}: Sunny, 72°F"

tools = [send_email, get_weather]
tools_by_name = {t.name: t for t in tools}

# Create LLM with tools
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
).bind_tools(tools)

print(f"Tools: {[t.name for t in tools]}")

In [ ]:
# State and nodes
class State(TypedDict):
    messages: Annotated[list, add_messages]

def agent_node(state: State) -> dict:
    """Call LLM to decide on action."""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

def tool_node(state: State) -> dict:
    """Execute tool calls."""
    outputs = []
    last_message = state["messages"][-1]
    
    for tc in last_message.tool_calls:
        print(f"  Executing: {tc['name']}({tc['args']})")
        result = tools_by_name[tc["name"]].invoke(tc["args"])
        outputs.append(ToolMessage(
            content=json.dumps(result),
            name=tc["name"],
            tool_call_id=tc["id"],
        ))
    
    return {"messages": outputs}

def should_continue(state: State) -> str:
    """Route based on tool calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

## Step 2: Add interrupt_before

The simplest way to add human review is `interrupt_before`. This pauses the graph **before** a node runs.

```python
graph = workflow.compile(
    checkpointer=memory,
    interrupt_before=["tools"]  # Pause before tools node
)
```

In [ ]:
# Build graph with interrupt_before
workflow = StateGraph(State)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent", should_continue, {"tools": "tools", "end": END}
)
workflow.add_edge("tools", "agent")

# Compile with interrupt_before
memory = MemorySaver()
graph = workflow.compile(
    checkpointer=memory,
    interrupt_before=["tools"]  # Pause before executing tools
)

print("Graph compiled with interrupt_before=['tools']")

## Step 3: Trigger an Interrupt

When the agent decides to call a tool, execution will pause:

In [ ]:
# Start a conversation that will trigger a tool call
thread_config = {"configurable": {"thread_id": "email-approval-1"}}

result = graph.invoke(
    {"messages": [("user", "Send an email to alice@example.com saying 'Hello from the agent!'")]},
    config=thread_config
)

print("Execution paused!")
print(f"Number of messages: {len(result['messages'])}")

In [ ]:
# Check what action is pending
state = graph.get_state(thread_config)

print(f"Next node to run: {state.next}")
print()

# Get the pending tool calls
last_message = state.values["messages"][-1]
if hasattr(last_message, "tool_calls") and last_message.tool_calls:
    print("Pending tool calls:")
    for tc in last_message.tool_calls:
        print(f"  Tool: {tc['name']}")
        print(f"  Args: {tc['args']}")
        print()

## Step 4: Review and Resume

At this point, a human can:
1. **Approve**: Resume execution with `None` (continue as-is)
2. **Reject**: Don't resume, end the conversation
3. **Modify**: Update state before resuming

In [ ]:
# APPROVE: Resume execution (pass None to continue)
print("Human approves the email...")

result = graph.invoke(None, config=thread_config)

print(f"\nFinal response: {result['messages'][-1].content}")

## Step 5: Rejection Example

Let's see what happens when we don't approve:

In [ ]:
# Start a new conversation
thread_config2 = {"configurable": {"thread_id": "email-approval-2"}}

result = graph.invoke(
    {"messages": [("user", "Send an email to boss@company.com with subject 'I quit!'")]},
    config=thread_config2
)

# Check pending action
state = graph.get_state(thread_config2)
last_msg = state.values["messages"][-1]

print("Pending action:")
for tc in last_msg.tool_calls:
    print(f"  {tc['name']}({tc['args']})")
print()
print("Human REJECTS this action!")
print("(Simply don't call invoke again - the thread is paused)")

## Step 6: Safe Actions (No Interrupt)

Not all tools need approval. Let's see what happens with a safe tool:

In [ ]:
# Weather is safe - still goes through interrupt
thread_config3 = {"configurable": {"thread_id": "weather-check"}}

result = graph.invoke(
    {"messages": [("user", "What's the weather in San Francisco?")]},
    config=thread_config3
)

# Check if interrupted
state = graph.get_state(thread_config3)
print(f"Next node: {state.next}")

if state.next:
    print("Graph is paused (even for safe tools with current config)")
    # Auto-approve weather checks
    result = graph.invoke(None, config=thread_config3)
    print(f"Result: {result['messages'][-1].content}")

## Step 7: Selective Interrupts

For more control, you can check the tool type and only interrupt for sensitive ones:

In [ ]:
# Define which tools are sensitive
SENSITIVE_TOOLS = {"send_email"}

def should_continue_selective(state: State) -> str:
    """Route based on tool type - only interrupt for sensitive tools."""
    last_message = state["messages"][-1]
    
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        # Check if any tool is sensitive
        for tc in last_message.tool_calls:
            if tc["name"] in SENSITIVE_TOOLS:
                return "sensitive_tools"  # Route to node with interrupt
        return "safe_tools"  # Route to node without interrupt
    return "end"

print(f"Sensitive tools: {SENSITIVE_TOOLS}")

In [ ]:
# Build selective interrupt graph
def sensitive_tool_node(state: State) -> dict:
    """Execute sensitive tools (with interrupt before)."""
    return tool_node(state)

def safe_tool_node(state: State) -> dict:
    """Execute safe tools (no interrupt)."""
    return tool_node(state)

workflow2 = StateGraph(State)
workflow2.add_node("agent", agent_node)
workflow2.add_node("sensitive_tools", sensitive_tool_node)
workflow2.add_node("safe_tools", safe_tool_node)

workflow2.add_edge(START, "agent")
workflow2.add_conditional_edges(
    "agent", 
    should_continue_selective, 
    {"sensitive_tools": "sensitive_tools", "safe_tools": "safe_tools", "end": END}
)
workflow2.add_edge("sensitive_tools", "agent")
workflow2.add_edge("safe_tools", "agent")

memory2 = MemorySaver()
graph2 = workflow2.compile(
    checkpointer=memory2,
    interrupt_before=["sensitive_tools"]  # Only interrupt sensitive
)

print("Selective interrupt graph compiled!")

In [ ]:
# Test: Weather (safe) - should NOT interrupt
thread_safe = {"configurable": {"thread_id": "safe-test"}}

result = graph2.invoke(
    {"messages": [("user", "What's the weather in London?")]},
    config=thread_safe
)

state = graph2.get_state(thread_safe)
if not state.next:
    print("Safe tool executed without interrupt!")
    print(f"Result: {result['messages'][-1].content}")
else:
    print(f"Unexpectedly interrupted at: {state.next}")

In [ ]:
# Test: Email (sensitive) - SHOULD interrupt
thread_sensitive = {"configurable": {"thread_id": "sensitive-test"}}

result = graph2.invoke(
    {"messages": [("user", "Email john@example.com with 'Meeting tomorrow'")]},
    config=thread_sensitive
)

state = graph2.get_state(thread_sensitive)
if state.next:
    print(f"Interrupted before: {state.next}")
    print("Human review required for sensitive action!")
else:
    print("Unexpected: did not interrupt")

## Complete Code: Approval Workflow

Here's a complete example with an approval helper:

In [ ]:
def run_with_approval(user_input: str, thread_id: str, auto_approve: bool = False):
    """Run agent with human approval workflow."""
    config = {"configurable": {"thread_id": thread_id}}
    
    # Initial run
    result = graph.invoke({"messages": [("user", user_input)]}, config=config)
    
    while True:
        state = graph.get_state(config)
        
        # Check if we're done
        if not state.next:
            break
        
        # Show pending actions
        last_msg = state.values["messages"][-1]
        if hasattr(last_msg, "tool_calls"):
            print("\n=== APPROVAL REQUIRED ===")
            for tc in last_msg.tool_calls:
                print(f"Action: {tc['name']}")
                print(f"Args: {tc['args']}")
            print("=========================")
        
        if auto_approve:
            print("Auto-approving...")
            result = graph.invoke(None, config=config)
        else:
            # In production, you'd get input from a UI
            approval = input("Approve? (y/n): ")
            if approval.lower() == 'y':
                result = graph.invoke(None, config=config)
            else:
                print("Action rejected!")
                break
    
    return result["messages"][-1].content

# Test with auto-approve
response = run_with_approval(
    "Send an email to test@example.com saying 'Test message'",
    "approval-demo",
    auto_approve=True
)
print(f"\nFinal: {response}")

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **interrupt_before** | Pause before a node runs |
| **interrupt_after** | Pause after a node runs |
| **state.next** | Shows which node will run next (None if done) |
| **invoke(None, config)** | Resume a paused graph |
| **get_state(config)** | Inspect current state |

## Common Patterns

1. **Approve all tools**: `interrupt_before=["tools"]`
2. **Selective approval**: Route to different nodes based on tool type
3. **Edit before execute**: Modify state before resuming

## What's Next?

In [Tutorial 05: Reflection](05_reflection.ipynb), you'll learn:
- Self-critique loops for quality improvement
- Generate → Reflect → Revise patterns
- Iterative refinement of outputs